**Problem Statement**: Predict HDB housing prices

# Importing

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import json
import glob
import pickle as pkl
import requests

In [2]:
# Import Data
# csv
property_info = pd.read_csv('hdb-property-information/hdb-property-information.csv') # hdb property information
resale_price_index = pd.read_csv('hdb-resale-price-index/housing-and-development-board-resale-price-index-1q2009-100-quarterly.csv') # resale price index
preschool_centre = pd.read_csv('listing-of-centres/listing-of-centres.csv', dtype={'postal_code': 'str'}) # preschool centre listing
school_info = pd.read_csv('school-directory-and-information/general-information-of-schools.csv', dtype={'postal_code': 'str'}) # primary and secondary school directory

# glob
resale_flat_prices = pd.concat([pd.read_csv(f) for f in glob.glob('resale-flat-prices/*.csv')])

In [3]:
# summary statistics function

def eda_clean(df):
    print('Dataset Statistics:\n')
    print(f'Shape of dataframe: {df.shape}')
    print('--------------------------------------')
    print(f'Null values in dataframe: {df.isna().sum().sum()}')
    print('--------------------------------------')
    print(f'% of Null values in dataframe: {round(((df.isna().sum().sum())/(df.shape[0])) * 100, 2)}%')
    print('--------------------------------------')
    print(f"Total duplicate rows: {df[df.duplicated()].shape[0]}")
    print('--------------------------------------')
    print(f"% duplicate rows: {round(df[df.duplicated()].shape[0] / df.shape[0] * 100, 2)}%")
    print('\nVariable Types')
    print(f"Columns Count: \n{df.dtypes.value_counts()}")

# Cleaning
## `Property Information`

In [4]:
eda_clean(property_info)

Dataset Statistics:

Shape of dataframe: (12472, 24)
--------------------------------------
Null values in dataframe: 0
--------------------------------------
% of Null values in dataframe: 0.0%
--------------------------------------
Total duplicate rows: 0
--------------------------------------
% duplicate rows: 0.0%

Variable Types
Columns Count: 
int64     15
object     9
dtype: int64


In [5]:
property_info.keys()

Index(['blk_no', 'street', 'max_floor_lvl', 'year_completed', 'residential',
       'commercial', 'market_hawker', 'miscellaneous', 'multistorey_carpark',
       'precinct_pavilion', 'bldg_contract_town', 'total_dwelling_units',
       '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold',
       'exec_sold', 'multigen_sold', 'studio_apartment_sold', '1room_rental',
       '2room_rental', '3room_rental', 'other_room_rental'],
      dtype='object')

In [6]:
property_info.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0


### Dropping Miscellaneous Column

In [7]:
# drop miscellaneous column
# I don't know what it means based on the data dictionary provided from source.
# I am assuming that this column will not significantly impact my predictive model.
property_info.drop(columns='miscellaneous', inplace=True)

### Choosing Blocks that have Residential Units

In [8]:
# drop rows whereby residential is N. 
# We are focusing on residential areas (which may include commercial units)
nil_residential = property_info[property_info['residential'] == 'N']
property_info.drop(nil_residential.index, inplace=True)

In [9]:
# checking the blocks that have 1 room rental.
property_info[property_info['1room_rental'] != 0]

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bldg_contract_town,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
12,1,HOLLAND CL,13,1973,Y,N,N,N,N,QT,...,0,0,0,0,0,0,319,0,0,0
63,10,NTH BRIDGE RD,16,1973,Y,Y,N,N,N,KWN,...,0,0,0,0,0,0,224,0,0,0
88,100,ALJUNIED CRES,12,1977,Y,N,N,N,N,GL,...,0,0,0,0,0,0,198,0,0,0
276,105,JLN BT MERAH,12,1967,Y,Y,N,N,N,BM,...,0,0,0,0,0,0,306,18,0,0
316,106,JLN BT MERAH,12,1967,Y,N,N,N,N,BM,...,0,0,0,0,0,0,312,18,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12337,967A,JURONG WEST ST 93,14,2017,Y,N,N,N,N,JW,...,0,0,0,0,0,0,91,130,0,0
12338,967B,JURONG WEST ST 93,14,2017,Y,N,N,N,N,JW,...,0,0,0,0,0,0,182,143,0,0
12404,986C,BUANGKOK CRES,16,2010,Y,N,N,N,N,HG,...,0,0,0,0,0,0,150,45,0,0
12406,986D,BUANGKOK CRES,16,2010,Y,N,N,N,N,HG,...,0,0,0,0,0,0,150,45,0,0


In [10]:
property_info.iloc[12]

# just zooming into the first row that I see above to view all the information under all the columns.

blk_no                           1
street                   JLN KUKOH
max_floor_lvl                   17
year_completed                1969
residential                      Y
commercial                       N
market_hawker                    Y
multistorey_carpark              N
precinct_pavilion                N
bldg_contract_town              CT
total_dwelling_units            84
1room_sold                       0
2room_sold                       0
3room_sold                       0
4room_sold                       0
5room_sold                       0
exec_sold                        0
multigen_sold                    0
studio_apartment_sold            0
1room_rental                     0
2room_rental                    56
3room_rental                    14
other_room_rental               14
Name: 14, dtype: object

In [11]:
property_info[property_info['studio_apartment_sold'] != 0]

# note that studio apartment is considered 2-room/ 2-room Flexi flat
# i didn't know that studio apartments exist in HDB, so there is a first for everything.

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bldg_contract_town,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
80,10B,BOON TIONG RD,30,2014,Y,N,N,N,N,BM,...,28,84,0,0,0,36,0,0,0,0
169,102A,DEPOT RD,21,2016,Y,N,N,N,N,BM,...,0,0,0,0,0,75,0,0,0,0
638,113C,MCNAIR RD,28,2016,Y,N,N,N,N,KWN,...,27,81,0,0,0,43,0,0,0,0
639,113D,MCNAIR RD,28,2016,Y,N,N,N,N,KWN,...,54,27,0,0,0,74,0,0,0,0
777,117,TAMPINES ST 11,11,2014,Y,N,N,N,N,TAP,...,0,0,0,0,0,125,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12100,91,TANGLIN HALT RD,40,2007,Y,N,N,N,N,QT,...,0,254,39,0,0,33,0,0,0,0
12137,92,DAWSON RD,43,2015,Y,N,N,N,N,QT,...,22,86,25,0,0,16,0,0,0,0
12144,92A,TELOK BLANGAH ST 31,30,2017,Y,N,N,N,N,BM,...,58,0,0,0,0,69,0,0,0,0
12145,92B,TELOK BLANGAH ST 31,30,2017,Y,N,N,N,N,BM,...,56,116,0,0,0,29,0,0,0,0


In [12]:
property_info.iloc[80]

blk_no                             101
street                   C'WEALTH CRES
max_floor_lvl                       10
year_completed                    1964
residential                          Y
commercial                           N
market_hawker                        N
multistorey_carpark                  N
precinct_pavilion                    N
bldg_contract_town                  QT
total_dwelling_units                76
1room_sold                           0
2room_sold                          10
3room_sold                          62
4room_sold                           4
5room_sold                           0
exec_sold                            0
multigen_sold                        0
studio_apartment_sold                0
1room_rental                         0
2room_rental                         0
3room_rental                         0
other_room_rental                    0
Name: 102, dtype: object

After iloc two specific rows, I find out that the 'total_dwelling_units' add up all the columns from x_sold and x_rental. So I will drop the x_sold and x_rental columns. In this scenario, we are not taking into consideration rental flats as they have a different price scheme. 

In [13]:
# wait. initially i thought rental flats and ownership flats do not exist in a certain block.
# that's not true. so we will have to minus off rental flats from total_dwelling_units.
# We are assuming that the presence of rental flats do not influence the surrounding resale flats.
rental = property_info[(property_info['1room_rental'] != 0) & (property_info['2room_rental'] != 0) & (property_info['3room_rental'] != 0) & (['other_room_rental'] != 0)]
rental.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bldg_contract_town,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
416,108,YISHUN RING RD,9,1981,Y,N,N,N,N,YS,...,137,32,0,0,0,0,16,16,7,0
435,109,BT BATOK WEST AVE 6,11,1984,Y,Y,N,N,N,BB,...,74,0,0,0,0,0,20,20,1,0
2753,204,TAMPINES ST 21,11,1982,Y,N,N,N,N,TAP,...,66,0,0,0,0,0,20,20,14,0
4931,313,HOUGANG AVE 5,13,1981,Y,Y,N,N,N,HG,...,78,0,0,0,0,0,24,48,30,0
4990,316,HOUGANG AVE 7,11,1981,Y,N,N,N,N,HG,...,81,0,0,0,0,0,10,20,24,0


In [14]:
# double check that the columns are in the right datatypes before i start...
property_info.dtypes

blk_no                   object
street                   object
max_floor_lvl             int64
year_completed            int64
residential              object
commercial               object
market_hawker            object
multistorey_carpark      object
precinct_pavilion        object
bldg_contract_town       object
total_dwelling_units      int64
1room_sold                int64
2room_sold                int64
3room_sold                int64
4room_sold                int64
5room_sold                int64
exec_sold                 int64
multigen_sold             int64
studio_apartment_sold     int64
1room_rental              int64
2room_rental              int64
3room_rental              int64
other_room_rental         int64
dtype: object

In [15]:
# So we will add up all the rental flat quantities and subtract that from total_dwelling_units.
property_info['rental_units'] = property_info['1room_rental'] + property_info['2room_rental'] + property_info['3room_rental'] + property_info['other_room_rental']

In [16]:
not_zero = property_info[property_info['rental_units'] != 0]
not_zero[['total_dwelling_units', 'rental_units']]

,total_dwelling_units,rental_units
7,247,247
12,319,319
14,84,84
15,244,1
19,179,2
...,...,...
12338,325,325
12404,195,195
12406,195,195
12430,108,3


In [17]:
# creating a column for hdb units ('own_units') that are purchased and not rented. 
# subtract no. of rented units from total dwelling units into 'own_units'
property_info['own_units'] = property_info['total_dwelling_units'] - property_info['rental_units']

In [18]:
property_info[['own_units', 'total_dwelling_units', 'rental_units']]

,own_units,total_dwelling_units,rental_units
0,142,142,0
1,206,206,0
3,102,102,0
4,55,55,0
5,96,96,0
...,...,...,...
12464,165,165,0
12467,185,185,0
12468,174,174,0
12470,0,306,306


### Dropping Other Columns

In [19]:
# now, we can drop 'total_dwelling_units' and 'rental_units'
property_info.drop(columns=['total_dwelling_units', 'rental_units'], inplace=True)

In [20]:
# making sure that we only want properties that have residential units
property_info['residential'].unique()

array(['Y'], dtype=object)

In [21]:
# drop irrelevant columns
property_info.drop(columns=['1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'exec_sold', 'multigen_sold', 'residential','studio_apartment_sold', 'year_completed'], inplace=True)

In [22]:
property_info.keys()

Index(['blk_no', 'street', 'max_floor_lvl', 'commercial', 'market_hawker',
       'multistorey_carpark', 'precinct_pavilion', 'bldg_contract_town',
       'own_units'],
      dtype='object')

### Imputing Y and N with 1 and 0

In [23]:
# impute 1 for Y and 0 for N.
property_info['commercial'] = property_info['commercial'].apply(lambda x: 1 if x == 'Y' else 0)
property_info['market_hawker'] = property_info['market_hawker'].apply(lambda x: 1 if x == 'Y' else 0)
property_info['precinct_pavilion'] = property_info['precinct_pavilion'].apply(lambda x: 1 if x == 'Y' else 0)
property_info['multistorey_carpark'] = property_info['multistorey_carpark'].apply(lambda x: 1 if x == 'Y' else 0)

### Create 'address' column (for concat + geolocation later)

In [24]:
# combine blk_no and street
property_info['address'] = property_info['blk_no'] + " " + property_info['street']
property_info.drop(columns=['blk_no', 'street'], inplace=True)

In [25]:
# one-hot encoding# We can exclude Unsepcified species from this selection
# towns_of_interest = ['AMK', 'BB', 'BD', 'BH', 'BM','BP','BT','CCK','CL','CT','GL','HG','JE','JW','KWN','MP','PG','PRC','QT','SB','SGN','SK','TAP','TG','TP','WL','YS']

# create loop to create dummified variables to indicate presence of Pipiens & Restuans species
# def dummify_towns(df):
#    for i in towns_of_interest:
#        df[i] = df['bldg_contract_town'].apply(lambda x: 1 if i in x else 0)
    
#    return df

In [26]:
# dum_prop_info = dummify_towns(property_info)

In [27]:
# dum_prop_info.drop(columns='bldg_contract_town', inplace=True)

## `Resale Price Index`

In [28]:
resale_price_index.head()

,quarter,index
0,1990-Q1,24.3
1,1990-Q2,24.4
2,1990-Q3,25.0
3,1990-Q4,24.7
4,1991-Q1,24.9


## `School Info`

In [29]:
school_info.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


### Drop Irrelevant Columns

In [30]:
# Drop irrelevant columns
school_info.drop(columns=['url_address','telephone_no','telephone_no_2', 'fax_no', 'fax_no_2', 'email_address','principal_name', 'first_vp_name', 'second_vp_name',
       'third_vp_name', 'fourth_vp_name', 'fifth_vp_name', 'sixth_vp_name'], inplace=True)

In [31]:
school_info.keys()

Index(['school_name', 'address', 'postal_code', 'mrt_desc', 'bus_desc',
       'dgp_code', 'zone_code', 'type_code', 'nature_code', 'session_code',
       'mainlevel_code', 'sap_ind', 'autonomous_ind', 'gifted_ind', 'ip_ind',
       'mothertongue1_code', 'mothertongue2_code', 'mothertongue3_code'],
      dtype='object')

### One-Hot Encoding

In [32]:
# one-hot encode 'zone_code', 'type_code', 'nature_code', 'session_code', 'mainlevel_code'
school_info = pd.get_dummies(data=school_info, columns=['zone_code', 'type_code', 'nature_code', 'session_code', 'mainlevel_code'], dtype=np.int64, drop_first=True)

In [33]:
# impute 1 for yes and 0 for No for 'sap_ind', 'autonomous_ind', 'gifted_ind', 'ip_ind'
school_info['sap_ind'] = school_info['sap_ind'].apply(lambda x: 1 if x == 'Yes' else 0)
school_info['autonomous_ind'] = school_info['autonomous_ind'].apply(lambda x: 1 if x == 'Yes' else 0)
school_info['gifted_ind'] = school_info['gifted_ind'].apply(lambda x: 1 if x == 'Yes' else 0)
school_info['ip_ind'] = school_info['ip_ind'].apply(lambda x: 1 if x == 'Yes' else 0)

In [34]:
# finding the unique values for each column
# one-hot encode 'zone_code', 'type_code', 'nature_code', 'session_code', 'mainlevel_code', 'mothertongue1_code', 'mothertongue2_code', 'mothertongue3_code'
print(f"mothertongue code 1 = {school_info['mothertongue1_code'].unique()}")
print(f"mothertongue code 2 = {school_info['mothertongue2_code'].unique()}")
print(f"mothertongue code 3 = {school_info['mothertongue3_code'].unique()}")

mothertongue code 1 = ['Chinese' 'na']
mothertongue code 2 = ['Malay' 'na']
mothertongue code 3 = ['Tamil' 'na']


In [35]:
school_info['mothertongue1_code'] = school_info['mothertongue1_code'].apply(lambda x: 1 if x == 'Chinese' else 0)
school_info['mothertongue2_code'] = school_info['mothertongue2_code'].apply(lambda x: 1 if x == 'Malay' else 0)
school_info['mothertongue3_code'] = school_info['mothertongue3_code'].apply(lambda x: 1 if x == 'Tamil' else 0)

## `Resale Flat Prices`

In [36]:
eda_clean(resale_flat_prices)

Dataset Statistics:

Shape of dataframe: (866131, 11)
--------------------------------------
Null values in dataframe: 709050
--------------------------------------
% of Null values in dataframe: 81.86%
--------------------------------------
Total duplicate rows: 1862
--------------------------------------
% duplicate rows: 0.21%

Variable Types
Columns Count: 
object     8
float64    2
int64      1
dtype: int64


In [37]:
resale_flat_prices.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
dtype: object

In [38]:
# a lot of the rows have the 'remaining_lease' as empty.
# we will need to do something with that.
print(f"non-empty 'remaining_lease' columns {resale_flat_prices[resale_flat_prices['remaining_lease'].notna()].shape}")
print(f"empty 'remaining_lease' columns {resale_flat_prices[resale_flat_prices['remaining_lease'].isna()].shape}")

non-empty 'remaining_lease' columns (157081, 11)
empty 'remaining_lease' columns (709050, 11)


In [39]:
resale_flat_prices.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'remaining_lease'],
      dtype='object')

In [40]:
resale_flat_prices.rename(columns={"month": "date"}, inplace=True)

In [41]:
# convert month to datetime
resale_flat_prices['month'] = pd.to_datetime(resale_flat_prices['date'], format='%Y/%m')
resale_flat_prices['year'] = pd.DatetimeIndex(resale_flat_prices['date']).year
resale_flat_prices['month'] = pd.DatetimeIndex(resale_flat_prices['date']).month

In [42]:
resale_flat_prices.head()

,date,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,month,year
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,3,2012
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN,3,2012
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN,3,2012
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN,3,2012
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN,3,2012


In [43]:
# combine block and street_name into one column, drop block and street_name
resale_flat_prices['address'] = resale_flat_prices['block'] + " " + resale_flat_prices['street_name']
resale_flat_prices.drop(columns=['block', 'street_name', 'town'], inplace=True)

In [44]:
resale_flat_prices.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [45]:
property_info.head()

,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bldg_contract_town,own_units,address
0,16,1,0,0,0,KWN,142,1 BEACH RD
1,14,0,0,0,0,BD,206,1 BEDOK STH AVE 1
3,15,0,0,0,0,BD,102,1 CHAI CHEE RD
4,4,1,0,0,0,PRC,55,1 CHANGI VILLAGE RD
5,25,0,0,0,0,BM,96,1 DELTA AVE


In [46]:
resale_flat_prices['remaining_lease'].unique()
# it's a little bit messy and a lot of the columns are empty.

array([nan, '61 years 04 months', '60 years 07 months',
       '62 years 05 months', '62 years 01 month', '63 years',
       '61 years 06 months', '58 years 04 months', '59 years 08 months',
       '59 years 06 months', '60 years', '62 years 08 months', '61 years',
       '60 years 10 months', '59 years 03 months', '61 years 05 months',
       '60 years 04 months', '62 years', '60 years 03 months',
       '63 years 09 months', '61 years 01 month', '61 years 10 months',
       '58 years 06 months', '59 years 04 months', '62 years 11 months',
       '60 years 08 months', '93 years 08 months', '93 years 07 months',
       '60 years 01 month', '94 years 08 months', '78 years 04 months',
       '60 years 06 months', '62 years 06 months', '58 years',
       '70 years 08 months', '63 years 04 months', '63 years 06 months',
       '67 years 07 months', '61 years 07 months', '68 years 02 months',
       '68 years 03 months', '56 years', '67 years 09 months',
       '67 years 05 months', '63 yea

In [47]:
# manually calculating the columns
resale_flat_prices['remaining_lease_years'] = resale_flat_prices['lease_commence_date'] + 98 - resale_flat_prices['year']

In [48]:
# drop 'remaining_lease' column
resale_flat_prices.drop(columns='remaining_lease', inplace=True)


In [49]:
print(resale_flat_prices.columns)

Index(['date', 'flat_type', 'storey_range', 'floor_area_sqm', 'flat_model',
       'lease_commence_date', 'resale_price', 'month', 'year', 'address',
       'remaining_lease_years'],
      dtype='object')


## `Preschool Centre`

In [50]:
preschool_centre.head()
# try to figure out how to include these features inside

,tp_code,centre_code,centre_name,organisation_code,organisation_description,service_model,centre_contact_no,centre_email_address,centre_address,postal_code,...,spark_certified,weekday_full_day,saturday,scheme_type,extended_operating_hours,provision_of_transport,government_subsidy,gst_regisration,last_updated,remarks
0,na,PT9814,MAPLEBEAR LITTLE FEATS PTE LTD,PT,Private Operators,CC,91521578,corporationplace@maplebear.sg,"2,CORPORATION ROAD,#01-13,CORPORATION PLACE,61...",618494,...,No,07:00-19:00,07:00-14:00,na,No,Yes,Yes,No,2021-12-09,Test remarks for row 1
1,na,PT9037,BERRY TREE PRESCHOOL PRIVATE LIMITED,PT,Private Operators,CC,68757517,YYWP5656@GMAIL.COM,"319C,ANCHORVALE DRIVE,#01-66,543319",543319,...,Yes,07:00-19:00,07:00-14:00,na,No,No,Yes,No,2021-12-09,Test remarks for row 2
2,na,YM0110,MY WORLD PRESCHOOL LTD.,YM,MY World Preschool Ltd,CC,67863590,tampinescentral@myworld.org.sg,"855,TAMPINES STREET 83,#01-262,520855",520855,...,Yes,07:00-19:00,07:00-14:00,Anchor Operator Scheme,No,No,Yes,Yes,2021-12-09,Test remarks for row 3
3,na,RC1856,Little Shepherds' Schoolhouse@Church of St Fra...,RC,Not-for-Profit Organisation,KN,62618212,lss-cosfajw.principal@catholicpreschool.edu.sg,"200,BOON LAY AVENUE,CHURCH OF ST. FRANCIS OF A...",649964,...,Yes,08:30-15:00,na,na,No,Yes,Yes,Yes,2021-12-09,Test remarks for row 4
4,na,PT9621,Brainy Bunch Pasir Ris Pte Ltd,PT,Private Operators,KN,97415166,murni@brainybunch.com,"273,PASIR RIS STREET 21,#01-498,510237",510237,...,No,07:30-17:30,na,na,No,Yes,Yes,No,2021-12-09,Test remarks for row 5


In [51]:
preschool_centre.reset_index(drop=True)

,tp_code,centre_code,centre_name,organisation_code,organisation_description,service_model,centre_contact_no,centre_email_address,centre_address,postal_code,...,spark_certified,weekday_full_day,saturday,scheme_type,extended_operating_hours,provision_of_transport,government_subsidy,gst_regisration,last_updated,remarks
0,na,PT9814,MAPLEBEAR LITTLE FEATS PTE LTD,PT,Private Operators,CC,91521578,corporationplace@maplebear.sg,"2,CORPORATION ROAD,#01-13,CORPORATION PLACE,61...",618494,...,No,07:00-19:00,07:00-14:00,na,No,Yes,Yes,No,2021-12-09,Test remarks for row 1
1,na,PT9037,BERRY TREE PRESCHOOL PRIVATE LIMITED,PT,Private Operators,CC,68757517,YYWP5656@GMAIL.COM,"319C,ANCHORVALE DRIVE,#01-66,543319",543319,...,Yes,07:00-19:00,07:00-14:00,na,No,No,Yes,No,2021-12-09,Test remarks for row 2
2,na,YM0110,MY WORLD PRESCHOOL LTD.,YM,MY World Preschool Ltd,CC,67863590,tampinescentral@myworld.org.sg,"855,TAMPINES STREET 83,#01-262,520855",520855,...,Yes,07:00-19:00,07:00-14:00,Anchor Operator Scheme,No,No,Yes,Yes,2021-12-09,Test remarks for row 3
3,na,RC1856,Little Shepherds' Schoolhouse@Church of St Fra...,RC,Not-for-Profit Organisation,KN,62618212,lss-cosfajw.principal@catholicpreschool.edu.sg,"200,BOON LAY AVENUE,CHURCH OF ST. FRANCIS OF A...",649964,...,Yes,08:30-15:00,na,na,No,Yes,Yes,Yes,2021-12-09,Test remarks for row 4
4,na,PT9621,Brainy Bunch Pasir Ris Pte Ltd,PT,Private Operators,KN,97415166,murni@brainybunch.com,"273,PASIR RIS STREET 21,#01-498,510237",510237,...,No,07:30-17:30,na,na,No,Yes,Yes,No,2021-12-09,Test remarks for row 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,TP0284,na,PCF Sparkletots Preschool @ Sembawang Central ...,ST,PAP Community Foundation,CC,0,GB.KN.309@pcf.org.sg,"309,Canberra Road,#01-109,750309",750309,...,No,07:00-19:00,07:00-14:00,Anchor Operator Scheme,No,No,Yes,Yes,2021-07-12,Test remarks for row 1930
1930,TP0283,na,PCF Sparkletots Preschool @ Nee Soon Link Blk ...,ST,PAP Community Foundation,CC,0,NL.KN.408@pcf.org.sg,"408,Yishun Avenue 6,#01-1264,760408",760408,...,No,07:00-19:00,07:00-14:00,Anchor Operator Scheme,No,No,Yes,Yes,2021-07-12,Test remarks for row 1931
1931,TP0285,na,PCF Sparkletots Preschool @ Admiralty Blk 581 ...,ST,PAP Community Foundation,CC,0,AD.DS.581@pcf.org.sg,"581,Woodlands Drive,#01-486,730581",730581,...,No,07:00-19:00,07:00-14:00,Anchor Operator Scheme,No,No,Yes,No,2021-07-12,Test remarks for row 1932
1932,TP0278,na,PCF Sparkletots Preschool @ Taman Jurong Blk 1...,ST,PAP Community Foundation,CC,0,PMD.CMS@pcf.org.sg,"150,Corporation Drive,610150",610150,...,No,07:00-19:00,07:00-14:00,Anchor Operator Scheme,No,No,Yes,No,2021-02-10,Test remarks for row 1933


In [52]:
eda_clean(preschool_centre)

Dataset Statistics:

Shape of dataframe: (1934, 29)
--------------------------------------
Null values in dataframe: 2
--------------------------------------
% of Null values in dataframe: 0.1%
--------------------------------------
Total duplicate rows: 0
--------------------------------------
% duplicate rows: 0.0%

Variable Types
Columns Count: 
object    28
int64      1
dtype: int64


In [53]:
preschool_centre.dtypes

tp_code                     object
centre_code                 object
centre_name                 object
organisation_code           object
organisation_description    object
service_model               object
centre_contact_no            int64
centre_email_address        object
centre_address              object
postal_code                 object
centre_website              object
infant_vacancy              object
pg_vacancy                  object
n1_vacancy                  object
n2_vacancy                  object
k1_vacancy                  object
k2_vacancy                  object
food_offered                object
second_languages_offered    object
spark_certified             object
weekday_full_day            object
saturday                    object
scheme_type                 object
extended_operating_hours    object
provision_of_transport      object
government_subsidy          object
gst_regisration             object
last_updated                object
remarks             

In [54]:
preschool_centre.keys()

Index(['tp_code', 'centre_code', 'centre_name', 'organisation_code',
       'organisation_description', 'service_model', 'centre_contact_no',
       'centre_email_address', 'centre_address', 'postal_code',
       'centre_website', 'infant_vacancy', 'pg_vacancy', 'n1_vacancy',
       'n2_vacancy', 'k1_vacancy', 'k2_vacancy', 'food_offered',
       'second_languages_offered', 'spark_certified', 'weekday_full_day',
       'saturday', 'scheme_type', 'extended_operating_hours',
       'provision_of_transport', 'government_subsidy', 'gst_regisration',
       'last_updated', 'remarks'],
      dtype='object')

In [55]:
# drop irrelevant columns
preschool_centre.drop(columns=['tp_code', 'centre_code', 'centre_contact_no',
       'centre_email_address', 'centre_website', 'last_updated', 'remarks'], inplace=True)

In [56]:
items = ['organisation_code', 'organisation_description', 'service_model', 'infant_vacancy',
        'pg_vacancy', 'n2_vacancy', 'k1_vacancy', 'k2_vacancy', 'food_offered', 'second_languages_offered',
        'spark_certified', 'weekday_full_day', 'saturday', 'scheme_type', 'extended_operating_hours', 'government_subsidy',
        'gst_regisration']

def description(input):
    for i in input:
        print(f"{i} = {preschool_centre[i].unique()}")

In [57]:
description(items)

organisation_code = ['PT' 'YM' 'RC' 'ST' 'PW' 'WP' 'NT' 'SK' 'EB' 'YW' 'KM']
organisation_description = ['Private Operators' 'MY World Preschool Ltd'
 'Not-for-Profit Organisation' 'PAP Community Foundation'
 'Presbyterian Community Services' 'Workplace'
 'NTUC First Campus Co-Operative Ltd' 'Skool4Kidz Pte Ltd'
 'E-Bridge Pre-school Pte. Ltd.'
 "Young Women's Christian Association of Singapore" 'MOE Kindergarten']
service_model = ['CC' 'KN' 'EYC-DS' 'DS' 'EYC']
infant_vacancy = ['na' 'Within 1 Year' 'More than 1 Year' 'Immediate']
pg_vacancy = ['na' 'Within 1 Year' 'Immediate' 'More than 1 Year']
n2_vacancy = ['na' 'Within 1 Year' 'Immediate' 'More than 1 Year']
k1_vacancy = ['na' 'Within 1 Year' 'Immediate' 'More than 1 Year']
k2_vacancy = ['na' 'Within 1 Year' 'Immediate' 'More than 1 Year']
food_offered = ['No Pork No Lard with No Beef (without Certification from MUIS but from Halal Sources)'
 'No Pork No Lard with No Beef (from Non-Halal Sources)' 'na'
 'Halal Food With No Beef (w

## Merging `property_info` and `resale_flat_prices`

In [58]:
# pd.merge(left = property_info,
#         right = resale_flat_prices,
#         on = "") 

df = resale_flat_prices.merge(property_info, how='left', left_on='address', right_on='address')

In [59]:
eda_clean(df)

Dataset Statistics:

Shape of dataframe: (864269, 18)
--------------------------------------
Null values in dataframe: 99085
--------------------------------------
% of Null values in dataframe: 11.46%
--------------------------------------
Total duplicate rows: 54
--------------------------------------
% duplicate rows: 0.01%

Variable Types
Columns Count: 
float64    8
object     6
int64      4
dtype: int64


In [60]:
df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [61]:
null = df[df.isnull().any(axis=1)]
# data that has null values is for those that do not have columns from property_info inside.
null.shape[0]/len(df)*100

1.6379026052544796

In [62]:
# about 14000 rows have null values. This makes up % of the dataset.
df.dropna(inplace=True)

In [63]:
df.head()

,date,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,month,year,address,remaining_lease_years,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bldg_contract_town,own_units
0,2012-03,2 ROOM,06 TO 10,45.0,Improved,1986,250000.0,3,2012,172 ANG MO KIO AVE 4,72,10.0,0.0,0.0,0.0,0.0,AMK,175.0
1,2012-03,2 ROOM,01 TO 05,44.0,Improved,1980,265000.0,3,2012,510 ANG MO KIO AVE 8,66,9.0,0.0,0.0,0.0,0.0,AMK,58.0
2,2012-03,3 ROOM,06 TO 10,68.0,New Generation,1980,315000.0,3,2012,610 ANG MO KIO AVE 4,66,11.0,0.0,0.0,0.0,0.0,AMK,258.0
3,2012-03,3 ROOM,01 TO 05,67.0,New Generation,1984,320000.0,3,2012,474 ANG MO KIO AVE 10,70,13.0,0.0,0.0,0.0,0.0,AMK,163.0
4,2012-03,3 ROOM,06 TO 10,67.0,New Generation,1980,321000.0,3,2012,604 ANG MO KIO AVE 5,66,12.0,0.0,0.0,0.0,0.0,AMK,151.0


In [64]:
df['flat_model'] = df['flat_model'].str.lower()
df['flat_type'] = df['flat_type'].str.lower()

In [65]:
print(df['flat_model'].unique())
df['flat_type'].unique()

['improved' 'new generation' 'standard' 'model a' 'simplified'
 'premium apartment' 'maisonette' 'model a-maisonette' 'apartment'
 'adjoined flat' 'multi generation' 'model a2' 'terrace'
 'improved-maisonette' 'premium maisonette' 'dbss' 'type s1' '2-room'
 'type s2' 'premium apartment loft']


array(['2 room', '3 room', '4 room', '5 room', 'executive',
       'multi-generation', '1 room', 'multi generation'], dtype=object)

In [66]:
df['flat_type'].replace('multi generation', 'multi-generation', inplace=True)

In [67]:
df.to_pickle('df.pkl')

### Dummifying

In [68]:
df_dum = pd.get_dummies(data=df, columns=['flat_type', 'flat_model', 'storey_range', 'bldg_contract_town'], dtype=np.int64, drop_first=True)

In [69]:
df_dum.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [70]:
print(eda_clean(df_dum))

Dataset Statistics:

Shape of dataframe: (850060, 88)
--------------------------------------
Null values in dataframe: 0
--------------------------------------
% of Null values in dataframe: 0.0%
--------------------------------------
Total duplicate rows: 0
--------------------------------------
% duplicate rows: 0.0%

Variable Types
Columns Count: 
int64      78
float64     8
object      2
dtype: int64
None


# Geocoding

## MRT Stations

In [71]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]

In [72]:
# function to find lat and long of each location
# mrt

mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)
    
    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_long.append('NotFound')
        print ("No Results")

Jurong East MRT Station,Lat: 1.33315281585758 Long: 103.742286332403
Bukit Batok MRT Station,Lat: 1.34903331201636 Long: 103.749566478309
Bukit Gombak MRT Station,Lat: 1.35861159094192 Long: 103.751790910733
Choa Chu Kang MRT Station,Lat: 1.38536316540225 Long: 103.744370779756
Yew Tee MRT Station,Lat: 1.39753506936297 Long: 103.747405150236
Kranji MRT Station,Lat: 1.42508698073648 Long: 103.762137459497
Marsiling MRT Station,Lat: 1.43252114855026 Long: 103.774074641403
Woodlands MRT Station,Lat: 1.43605761708128 Long: 103.787938777173
Admiralty MRT Station,Lat: 1.44058856161847 Long: 103.800990519771
Sembawang MRT Station,Lat: 1.44905082158502 Long: 103.820046140211
Canberra MRT Station,Lat: 1.44307664075699 Long: 103.829702590959
Yishun MRT Station,Lat: 1.42944308477331 Long: 103.835005047246
Khatib MRT Station,Lat: 1.41738337009565 Long: 103.832979908243
Yio Chu Kang MRT Station,Lat: 1.38175587099132 Long: 103.84494727118
Ang Mo Kio MRT Station,Lat: 1.36993284962264 Long: 103.849558

In [73]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

## HDB Addresses

In [74]:
address_list = df['address'].unique()

In [75]:
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0

for row in range(len(address_list)): 
    query_address = address_list[row]
    query_string='https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)          

     
    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
        print (str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
    else:
        print ("No Results")

172 ANG MO KIO AVE 4 ,Lat: 1.37469423247424 Long: 103.836462867365
510 ANG MO KIO AVE 8 ,Lat: 1.37340092645025 Long: 103.849073244454
610 ANG MO KIO AVE 4 ,Lat: 1.37939470300077 Long: 103.839157232686
474 ANG MO KIO AVE 10 ,Lat: 1.36275784702216 Long: 103.858015323667
604 ANG MO KIO AVE 5 ,Lat: 1.37986658053966 Long: 103.835976650638
154 ANG MO KIO AVE 5 ,Lat: 1.37626067387977 Long: 103.841230957516
110 ANG MO KIO AVE 4 ,Lat: 1.37045937010888 Long: 103.837419259859
445 ANG MO KIO AVE 10 ,Lat: 1.36698004515058 Long: 103.85571808973
476 ANG MO KIO AVE 10 ,Lat: 1.36238829184688 Long: 103.857880564737
631 ANG MO KIO AVE 4 ,Lat: 1.3795572819161 Long: 103.840735651035
155 ANG MO KIO AVE 4 ,Lat: 1.37587723828643 Long: 103.840067482919
560 ANG MO KIO AVE 10 ,Lat: 1.37081626777721 Long: 103.859192334256
561 ANG MO KIO AVE 10 ,Lat: 1.37057765782635 Long: 103.857854578394
405 ANG MO KIO AVE 10 ,Lat: 1.36157912717139 Long: 103.853805386957
548 ANG MO KIO AVE 10 ,Lat: 1.37291376311161 Long: 103.857

4 BEDOK STH AVE 1 ,Lat: 1.32248962296588 Long: 103.933405282585
424 BEDOK NTH AVE 1 ,Lat: 1.32842001017396 Long: 103.927482674529
158 BEDOK STH AVE 3 ,Lat: 1.31867038706863 Long: 103.945105611031
616 BEDOK RESERVOIR RD ,Lat: 1.33182713238417 Long: 103.914991045591
55 CHAI CHEE DR ,Lat: 1.32684846948333 Long: 103.919015573079
536 BEDOK NTH ST 3 ,Lat: 1.3335411894605 Long: 103.923190774432
57 NEW UPP CHANGI RD ,Lat: 1.32424554153984 Long: 103.941568704132
104 BEDOK NTH AVE 4 ,Lat: 1.3332985212474 Long: 103.936601473643
92 BEDOK NTH AVE 4 ,Lat: 1.33244414877355 Long: 103.941297978178
701 BEDOK RESERVOIR RD ,Lat: 1.33764156135414 Long: 103.918906218067
103 BEDOK RESERVOIR RD ,Lat: 1.32877022232019 Long: 103.909381354431
218 BEDOK NTH ST 1 ,Lat: 1.32735657259634 Long: 103.933767796194
402 BEDOK NTH AVE 3 ,Lat: 1.33055472908632 Long: 103.933686723902
102 BEDOK NTH AVE 4 ,Lat: 1.33390654765002 Long: 103.937458398204
116 BEDOK NTH RD ,Lat: 1.33100823526591 Long: 103.938066838998
425 BEDOK NTH 

507 BT BATOK ST 52 ,Lat: 1.3522210432368 Long: 103.753979892768
413 BT BATOK WEST AVE 4 ,Lat: 1.36356572674155 Long: 103.746596967526
422 BT BATOK WEST AVE 2 ,Lat: 1.36218905539241 Long: 103.745774940015
374 BT BATOK ST 31 ,Lat: 1.35798291106742 Long: 103.750736425731
416 BT BATOK WEST AVE 4 ,Lat: 1.36323334513612 Long: 103.745988532649
501 BT BATOK ST 52 ,Lat: 1.35149319356169 Long: 103.754998121996
342 BT BATOK ST 34 ,Lat: 1.36258323081948 Long: 103.748574945077
272 BT BATOK EAST AVE 4 ,Lat: 1.34881544166238 Long: 103.757951913114
367 BT BATOK ST 31 ,Lat: 1.36033274170697 Long: 103.749691516072
347 BT BATOK ST 34 ,Lat: 1.36227034200422 Long: 103.75119293891
312 BT BATOK ST 32 ,Lat: 1.35987875096146 Long: 103.74733565667
175 BT BATOK WEST AVE 8 ,Lat: 1.34554897953938 Long: 103.741598629749
421 BT BATOK WEST AVE 2 ,Lat: 1.36251752917877 Long: 103.744946684475
245 BT BATOK EAST AVE 5 ,Lat: 1.3496481268301 Long: 103.757332776738
504 BT BATOK ST 52 ,Lat: 1.35196560331027 Long: 103.7549836

26D JLN MEMBINA ,Lat: 1.28209883987158 Long: 103.825202810934
13 CANTONMENT CL ,Lat: 1.27469944029956 Long: 103.840485373883
51 TELOK BLANGAH DR ,Lat: 1.27351337938811 Long: 103.810145340012
77 TELOK BLANGAH DR ,Lat: 1.27394153200448 Long: 103.808817449676
44 MOH GUAN TER ,Lat: 1.28442020673303 Long: 103.831056918196
14A TELOK BLANGAH CRES ,Lat: 1.27776030069589 Long: 103.820730062207
55 TELOK BLANGAH DR ,Lat: 1.27396036035569 Long: 103.812149525435
109B DEPOT RD ,Lat: 1.28099937159398 Long: 103.809225279521
3 DELTA AVE ,Lat: 1.29169833480355 Long: 103.827334523132
50 TELOK BLANGAH DR ,Lat: 1.27309040956353 Long: 103.810143564301
88 TELOK BLANGAH HTS ,Lat: 1.27736919004889 Long: 103.808254924084
11 CANTONMENT CL ,Lat: 1.27442604746069 Long: 103.839671514269
115 BT PURMEI RD ,Lat: 1.27535201369639 Long: 103.826950180582
118D JLN MEMBINA ,Lat: 1.2826810505173 Long: 103.826583442179
5 DELTA AVE ,Lat: 1.29223145125728 Long: 103.827600712862
39 TELOK BLANGAH RISE ,Lat: 1.27174724526041 Long

472 CHOA CHU KANG AVE 3 ,Lat: 1.37908544061279 Long: 103.737542449427
152 JLN TECK WHYE ,Lat: 1.37881790510246 Long: 103.758023837892
686C CHOA CHU KANG CRES ,Lat: 1.40468369607815 Long: 103.74900201661
484D CHOA CHU KANG AVE 5 ,Lat: 1.37534797754277 Long: 103.737317097286
505 CHOA CHU KANG ST 51 ,Lat: 1.38907418639043 Long: 103.742140885339
289 CHOA CHU KANG AVE 3 ,Lat: 1.37936640828773 Long: 103.740771937699
486 CHOA CHU KANG AVE 5 ,Lat: 1.37544982404192 Long: 103.738349548996
658 CHOA CHU KANG CRES ,Lat: 1.40075418778953 Long: 103.749259984055
296 CHOA CHU KANG AVE 2 ,Lat: 1.37738338043104 Long: 103.740781439449
459 CHOA CHU KANG AVE 4 ,Lat: 1.37818434118826 Long: 103.735126609036
206 CHOA CHU KANG CTRL ,Lat: 1.3806670668767 Long: 103.748339853034
446 CHOA CHU KANG AVE 4 ,Lat: 1.3807129959352 Long: 103.73689142472
295 CHOA CHU KANG AVE 2 ,Lat: 1.37816643855629 Long: 103.740967962566
485A CHOA CHU KANG AVE 5 ,Lat: 1.37461810751349 Long: 103.738364995209
210 CHOA CHU KANG CTRL ,Lat: 1

...

432C YISHUN AVE 1 ,Lat: 1.41949570230706 Long: 103.845973427257
175C YUNG KUANG RD ,Lat: 1.33278786165574 Long: 103.721640089539
175A YUNG KUANG RD ,Lat: 1.33193786368443 Long: 103.721561508725
527A PASIR RIS ST 51 ,Lat: 1.36842293498041 Long: 103.947593701801
313C SUMANG LINK ,Lat: 1.41062885103467 Long: 103.901551852458
453A FERNVALE RD ,Lat: 1.39447883247668 Long: 103.875558391505
331C ANCHORVALE ST ,Lat: 1.39557791060487 Long: 103.889773699699
279C SENGKANG EAST AVE ,Lat: 1.38572932197005 Long: 103.892595434586
326D ANCHORVALE RD ,Lat: 1.39613052938418 Long: 103.888097116033
333A ANCHORVALE LINK ,Lat: 1.39468433299355 Long: 103.89086832093
334B ANCHORVALE CRES ,Lat: 1.39846172684413 Long: 103.889969214694
868C TAMPINES AVE 8 ,Lat: 1.35585780112316 Long: 103.933334387633
523B TAMPINES CTRL 7 ,Lat: 1.35986668073238 Long: 103.938372890966
523C TAMPINES CTRL 7 ,Lat: 1.35988670789408 Long: 103.938870018657
342B YISHUN RING RD ,Lat: 1.43024596089696 Long: 103.844838329615
432B YISHUN AVE

476A UPP SERANGOON VIEW ,Lat: 1.37676038157719 Long: 103.904050364711
477A UPP SERANGOON VIEW ,Lat: 1.37830875820742 Long: 103.903841957906
475D UPP SERANGOON CRES ,Lat: 1.37518254401411 Long: 103.900405982824
138C YUAN CHING RD ,Lat: 1.32843797727643 Long: 103.723795893377
138B YUAN CHING RD ,Lat: 1.32841510984309 Long: 103.724304147765
524C PASIR RIS ST 51 ,Lat: 1.37054529360258 Long: 103.947594575271
673C EDGEFIELD PLAINS ,Lat: 1.40722159434779 Long: 103.910591907295
261B PUNGGOL WAY ,Lat: 1.40601534722271 Long: 103.896508270967
666B PUNGGOL DR ,Lat: 1.40179672508929 Long: 103.915278092564
416B FERNVALE LINK ,Lat: 1.39048519034443 Long: 103.880509545623
414A FERNVALE LINK ,Lat: 1.38958310807082 Long: 103.87947708432
494E TAMPINES ST 43 ,Lat: 1.36421531545345 Long: 103.953511202271
180A MARSILING RD ,Lat: 1.43904320861563 Long: 103.778109726656
505A YISHUN ST 51 ,Lat: 1.41884459566005 Long: 103.84482453185
748B BEDOK RESERVOIR CRES ,Lat: 1.33629391354583 Long: 103.921242294579
747C B

442B FAJAR RD ,Lat: 1.38143384735951 Long: 103.771366304235
442C FAJAR RD ,Lat: 1.38148756854256 Long: 103.770942229271
18A CIRCUIT RD ,Lat: 1.32932082372457 Long: 103.888071748726
316 HOUGANG AVE 7 ,Lat: 1.36497584742043 Long: 103.896809883055
57 TEBAN GDNS RD ,Lat: 1.32254226544254 Long: 103.737560699176
256C SUMANG WALK ,Lat: 1.40540203681824 Long: 103.895910481206
256D SUMANG WALK ,Lat: 1.40494870154993 Long: 103.896007686194
678A PUNGGOL DR ,Lat: 1.40566213236302 Long: 103.908718985826
87 DAWSON RD ,Lat: 1.2957304268451 Long: 103.809759937443
868B TAMPINES AVE 8 ,Lat: 1.35592432083149 Long: 103.932912367489
440B CLEMENTI AVE 3 ,Lat: 1.31623936538205 Long: 103.763566375801
376C HOUGANG ST 32 ,Lat: 1.36406322514465 Long: 103.893679750644
8A UPP BOON KENG RD ,Lat: 1.31291757072948 Long: 103.869743094328
530B PASIR RIS DR 1 ,Lat: 1.37168953845762 Long: 103.949502101347
530A PASIR RIS DR 1 ,Lat: 1.37114053931458 Long: 103.949850548422
668A EDGEFIELD PLAINS ,Lat: 1.40501880052452 Long: 

312C CLEMENTI AVE 4 ,Lat: 1.31952267914453 Long: 103.764025116013
119 ALJUNIED AVE 2 ,Lat: 1.31920342964443 Long: 103.886335477819
52 CASSIA CRES ,Lat: 1.31067221805457 Long: 103.88503171279
63 SIMS PL ,Lat: 1.31659507864197 Long: 103.879545641729
313 HOUGANG AVE 5 ,Lat: 1.36521740546842 Long: 103.894771265948
932B HOUGANG AVE 9 ,Lat: 1.37512666113384 Long: 103.882234927883
932A HOUGANG AVE 9 ,Lat: 1.37526558111202 Long: 103.881750957344
933A HOUGANG AVE 9 ,Lat: 1.37575796713264 Long: 103.882032792636
473B UPP SERANGOON CRES ,Lat: 1.37593845080051 Long: 103.901084524846
473A UPP SERANGOON CRES ,Lat: 1.37682525094625 Long: 103.900641038235
473C UPP SERANGOON CRES ,Lat: 1.37628006481899 Long: 103.901387559
364B UPP SERANGOON RD ,Lat: 1.37051434802161 Long: 103.897132658546
365C UPP SERANGOON RD ,Lat: 1.36928451336933 Long: 103.89713459868
365A UPP SERANGOON RD ,Lat: 1.36971560040268 Long: 103.897624425125
365D UPP SERANGOON RD ,Lat: 1.3696172178659 Long: 103.896612783518
365B UPP SERANGO

121C CANBERRA ST ,Lat: 1.44744539468374 Long: 103.834503117404
672B YISHUN AVE 4 ,Lat: 1.41859080728129 Long: 103.841823925238
813A YISHUN RING RD ,Lat: 1.41824920330568 Long: 103.832595236905
502D YISHUN ST 51 ,Lat: 1.41733258969906 Long: 103.840861112656
721 ANG MO KIO AVE 8 ,Lat: 1.37234984781557 Long: 103.847899479721
702 ANG MO KIO AVE 8 ,Lat: 1.369767934422 Long: 103.84757946328
158 ANG MO KIO AVE 4 ,Lat: 1.3747805874721 Long: 103.838997571585
340 ANG MO KIO AVE 1 ,Lat: 1.36445049902521 Long: 103.849464012145
727 CLEMENTI WEST ST 2 ,Lat: 1.30413678048377 Long: 103.76381785742
721 CLEMENTI WEST ST 2 ,Lat: 1.3024620264944 Long: 103.764555289938
25 NEW UPP CHANGI RD ,Lat: 1.3232501334952 Long: 103.933835585816


## Pre-School

In [76]:
list_of_preschool = preschool_centre['postal_code']

In [77]:
preschool_centre.loc[1131,['postal_code','centre_address']] = [399913,'1 GUILLEMARD CRES']

In [78]:
# function to find lat and long of each location
# pre-school

preschool_lat = []
preschool_long = []

for i in range(0, len(list_of_preschool)):
    query_address = list_of_preschool[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_preschool=json.loads(resp.content)
    
    if data_preschool['found'] != 0:
        preschool_lat.append(float(data_preschool["results"][0]["LATITUDE"]))
        preschool_long.append(float(data_preschool["results"][0]["LONGITUDE"]))

        print (str(query_address)+",Lat: "+data_preschool['results'][0]['LATITUDE'] +" Long: "+data_preschool['results'][0]['LONGITUDE'])

    else:
        preschool_long.append('NotFound')
        preschool_lat.append('NotFound')
        print (str(query_address)+ "No Results")

618494,Lat: 1.32454068043523 Long: 103.720165889611
543319,Lat: 1.39088901071724 Long: 103.89063413206
520855,Lat: 1.35358250009189 Long: 103.935299480466
649964,Lat: 1.34804040348048 Long: 103.711184664543
510237,Lat: 1.37407916915993 Long: 103.962867929916
560533,Lat: 1.37416923724326 Long: 103.853001920124
640642,Lat: 1.34036989694199 Long: 103.695839824959
641665,Lat: 1.34009319246619 Long: 103.703124305023
520942,Lat: 1.34977339661508 Long: 103.940622244099
530321,Lat: 1.36710131174973 Long: 103.898278016767
548651,Lat: 1.39168887721389 Long: 103.900386351553
460431,Lat: 1.32922126388158 Long: 103.932502461445
467624,Lat: 1.31834303213618 Long: 103.941364027549
760411,Lat: 1.42574178895795 Long: 103.847404279075
536290,Lat: 1.34580622499118 Long: 103.875741690705
418929,Lat: 1.31963983510783 Long: 103.912912683775
459210,Lat: 1.30974523556234 Long: 103.923485025317
307987,Lat: 1.31978249878705 Long: 103.842142960008
650345,Lat: 1.36284881503816 Long: 103.750843242288
520411,Lat: 1

530606,Lat: 1.3706773440242 Long: 103.885941015073
538840,Lat: 1.36475744073534 Long: 103.892622237823
538766,Lat: 1.37253315637342 Long: 103.893883460466
535736,Lat: 1.37120180912797 Long: 103.898290783389
530434,Lat: 1.37647221623833 Long: 103.893822106133
532992,Lat: 1.3838267304931 Long: 103.881672473881
530246,Lat: 1.35876258437852 Long: 103.891460074318
534908,Lat: 1.34304827970778 Long: 103.88221422444
538875,Lat: 1.35885665114265 Long: 103.889428375752
530127,Lat: 1.35291722123534 Long: 103.88567990649
539830,Lat: 1.35474020649844 Long: 103.883092839562
534625,Lat: 1.35453914024315 Long: 103.878126695986
536186,Lat: 1.34656520353906 Long: 103.88279441469
530997,Lat: 1.38544284304831 Long: 103.880763249489
530950,Lat: 1.37559305335225 Long: 103.878761073257
530706,Lat: 1.36596036769891 Long: 103.888646476566
530677,Lat: 1.37112901642432 Long: 103.8843188702
530667,Lat: 1.37378501535751 Long: 103.883849096326
530528,Lat: 1.37511919152466 Long: 103.890726071611
530170,Lat: 1.35146

760865,Lat: 1.41426758730393 Long: 103.838259938154
437738,Lat: 1.30043482264195 Long: 103.886613095098
469538,Lat: 1.3318149698608 Long: 103.947573704085
683817,Lat: 1.37756652387037 Long: 103.748989532974
597627,Lat: 1.31984371963095 Long: 103.779326736396
298135,Lat: 1.3336502448472 Long: 103.839419152835
328827,Lat: 1.32533672784378 Long: 103.853993102406
550407,Lat: 1.34718613034606 Long: 103.872968182223
160054,Lat: 1.28743641379469 Long: 103.841175875514
679331,Lat: 1.36778358876186 Long: 103.768107426457
397699,Lat: 1.30121118638528 Long: 103.872905191992
367984,Lat: 1.33356251802176 Long: 103.88638828999
390052,Lat: 1.31067221805457 Long: 103.88503171279
760311,Lat: 1.43157506974497 Long: 103.840815309067
609921,Lat: 1.32910540395565 Long: 103.746173268778
540131,Lat: 1.38934424778155 Long: 103.903081571286
769372,Lat: 1.43902187465443 Long: 103.828050229606
759130,Lat: 1.4259064181226 Long: 103.823772312405
609966,Lat: 1.32852674067117 Long: 103.752854514418
642662,Lat: 1.337

320085,Lat: 1.32278920327274 Long: 103.856444472629
541116,Lat: 1.38246038039285 Long: 103.902238681998
534934,Lat: 1.34425558746427 Long: 103.881861642477
469332,Lat: 1.3219367154385 Long: 103.931534884849
486046,Lat: 1.34418809031642 Long: 103.965001591021
279050,Lat: 1.31367483143637 Long: 103.783180618711
509397,Lat: 1.37746857655612 Long: 103.958408733539
598754,Lat: 1.3398938388841 Long: 103.766610017795
138639,Lat: 1.30541103549045 Long: 103.788371152389
128583,Lat: 1.30642089064512 Long: 103.767001510641
640518,Lat: 1.34508681384068 Long: 103.717978485296
427664,Lat: 1.30689281661852 Long: 103.904582811321
470742,Lat: 1.33808524960901 Long: 103.922090734567
449283,Lat: 1.30985532852467 Long: 103.927555067646
249537,Lat: 1.30444957282438 Long: 103.812222946283
805946,Lat: 1.39049673555442 Long: 103.854010727913
670545,Lat: 1.38159642060888 Long: 103.763496336507
520433,Lat: 1.35904050139779 Long: 103.951132576329
129605,Lat: 1.30264556890412 Long: 103.764832072851
760925,Lat: 1.

456929,Lat: 1.31680131547259 Long: 103.918409414644
828717,Lat: 1.40185430270244 Long: 103.910410112245
467961,Lat: 1.31973066955085 Long: 103.952086797844
640409,Lat: 1.35409792704061 Long: 103.725324736878
278886,Lat: 1.31396271756411 Long: 103.779998327203
737854,Lat: 1.43502365592364 Long: 103.804760536879
359826,Lat: 1.34255471804221 Long: 103.862271391697
822409,Lat: 1.41758014529069 Long: 103.902672012959
329979,Lat: 1.3290393738761 Long: 103.848481401952
428918,Lat: 1.30760256463236 Long: 103.907078176022
389801,Lat: 1.31385763534938 Long: 103.882241892183
387606,Lat: 1.31478309958573 Long: 103.879130196072
278967,Lat: 1.31025541884853 Long: 103.795350439947
159049,Lat: 1.28978712200723 Long: 103.814448344993
528584,Lat: 1.35027065611043 Long: 103.926893506073
760703,Lat: 1.42958489778701 Long: 103.828851112708
732687,Lat: 1.44188539470704 Long: 103.807161942037
307965,Lat: 1.31675907634497 Long: 103.841125998933
309007,Lat: 1.31930872020019 Long: 103.832058809956
348475,Lat: 1

...

819663,Lat: 1.35640628728788 Long: 103.986211815854
609930,Lat: 1.33028201498891 Long: 103.747247200113
138632,Lat: 1.2995985718601 Long: 103.787640882653
609923,Lat: 1.32696436520813 Long: 103.746250890926
609607,Lat: 1.33590457626914 Long: 103.744636416564
510258,Lat: 1.36706634197242 Long: 103.964635610477
651395,Lat: 1.36448974299463 Long: 103.751987145985
349844,Lat: 1.33440986402149 Long: 103.875537215276
387908,Lat: 1.31598945461254 Long: 103.889448079778
330005,Lat: 1.31629995826281 Long: 103.860397463054
341115,Lat: 1.33645283610396 Long: 103.87276093968
486035,Lat: 1.33787507957089 Long: 103.964469745735
117488,Lat: 1.27871081603146 Long: 103.788327649564
658065,Lat: 1.33696571508317 Long: 103.759665479168
188596,Lat: 1.29646823225185 Long: 103.855547492366
439291,Lat: 1.29970048930189 Long: 103.893880704979
554360,Lat: 1.37598331035084 Long: 103.870647299542
439659,Lat: 1.30236020335452 Long: 103.896445900986
651296,Lat: 1.34409358218354 Long: 103.746409176377
270024,Lat: 1.

680276,Lat: 1.37956864583538 Long: 103.744327184637
547686,Lat: 1.39139752452014 Long: 103.900374331539
680427,Lat: 1.38088419447891 Long: 103.73976352832
684686,Lat: 1.40411744569373 Long: 103.748355036188
680542,Lat: 1.39360501825299 Long: 103.744792619365
640641,Lat: 1.34066811398303 Long: 103.696356784294
679675,Lat: 1.37346487819206 Long: 103.76750951079
544618,Lat: 1.39838110843618 Long: 103.887327345481
560556,Lat: 1.37203236308683 Long: 103.857625375797
570409,Lat: 1.36221019788509 Long: 103.834946047117
541330,Lat: 1.39662723563942 Long: 103.889865427006
410116,Lat: 1.32278533070059 Long: 103.911863013412
461032,Lat: 1.32371728387748 Long: 103.92500108451
400326,Lat: 1.32717162158898 Long: 103.903804176812
400341,Lat: 1.32671260607254 Long: 103.901560745645
540351,Lat: 1.39332786581517 Long: 103.886617625113
560319,Lat: 1.36511023350774 Long: 103.847379037914
561310,Lat: 1.36481255187585 Long: 103.843844243986
550517,Lat: 1.37132342766086 Long: 103.874085635267
460409,Lat: 1.3

760201,Lat: 1.43064865606425 Long: 103.837996781156
760742,Lat: 1.42972006809811 Long: 103.83310944896
768019,Lat: 1.42875460572994 Long: 103.836324536111
760626,Lat: 1.41749366890869 Long: 103.835749017424
730424,Lat: 1.42728730939008 Long: 103.774859398322
520154,Lat: 1.34480692542205 Long: 103.956996641927
520165,Lat: 1.34553016120065 Long: 103.959248804134
530125,Lat: 1.35240984227777 Long: 103.886491570367
650512,Lat: 1.35398085110663 Long: 103.752778228297
120714,Lat: 1.30430341306691 Long: 103.761682084014
522606,Lat: 1.36230165817252 Long: 103.937401758053
548593,Lat: 1.39122301260171 Long: 103.902038578713
570505,Lat: 1.34891430036517 Long: 103.848662284887
677899,Lat: 1.38018163022688 Long: 103.76041930548
309763,Lat: 1.31763329946712 Long: 103.839896397053
463807,Lat: 1.3279979407231 Long: 103.92324860169
521524,Lat: 1.35835090076118 Long: 103.938908850193
425500,Lat: 1.31182050839719 Long: 103.910557234076
640478,Lat: 1.34735317434247 Long: 103.723952185676
739983,Lat: 1.44

416923,Lat: 1.32147757536992 Long: 103.909863907893
520803,Lat: 1.34519808450169 Long: 103.937688607467
129781,Lat: 1.29969494386434 Long: 103.768262238489
068815,Lat: 1.27743927179096 Long: 103.848687622624
460520,Lat: 1.33046461779259 Long: 103.92813426172
440089,Lat: 1.30129037884726 Long: 103.906458593236
423731,Lat: 1.30650535851357 Long: 103.905328960052
287994,Lat: 1.33759373277131 Long: 103.793707423659
329983,Lat: 1.32710813506067 Long: 103.846374256983
238698,Lat: 1.29731102645565 Long: 103.843201274544
142059,Lat: 1.29318964398969 Long: 103.81125258838
429063,Lat: 1.30956753585429 Long: 103.913675410417
544709,Lat: 1.39019431975813 Long: 103.897033070016
138628,Lat: 1.29806429260644 Long: 103.790115604433
099982,Lat: 1.24840314105618 Long: 103.82840268366
380128,Lat: 1.31830919498771 Long: 103.887754005604
458493,Lat: 1.319768568322 Long: 103.921356987677
460172,Lat: 1.32066917100909 Long: 103.94468952936
550335,Lat: 1.3494422140895 Long: 103.871067807837
760288,Lat: 1.43994

510197,Lat: 1.36640299690077 Long: 103.960738659652
688892,Lat: 1.38115420648197 Long: 103.751907169203
730664,Lat: 1.43619845734491 Long: 103.797798851368
339940,Lat: 1.31626605933927 Long: 103.862529469668
339949,Lat: 1.31691578544271 Long: 103.863109192276
340101,Lat: 1.33623671562033 Long: 103.871287596113
520713,Lat: 1.35924098095519 Long: 103.937646641338
822422,Lat: 1.4198034954577 Long: 103.905195322462
640642,Lat: 1.34036989694199 Long: 103.695839824959
641665,Lat: 1.34009319246619 Long: 103.703124305023
510517,Lat: 1.37468101222365 Long: 103.943850333671
520261,Lat: 1.35430349274332 Long: 103.950965475574
560533,Lat: 1.37416923724326 Long: 103.853001920124
750309,Lat: 1.4467573459998 Long: 103.821562407206
760408,Lat: 1.42626632416059 Long: 103.848703212672
730581,Lat: 1.42989030003699 Long: 103.796403905753
610150,Lat: 1.32724211434074 Long: 103.722352699939
828603,Lat: 1.4079458570585 Long: 103.909436431264


In [79]:
# find the 'NotFound' and impute manually

## Secondary/Primary School

In [80]:
list_of_school = school_info['postal_code']

In [81]:
# function to find lat and long of each location
# school

school_lat = []
school_long = []

for i in range(0, len(list_of_school)):
    query_address = list_of_school[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_school=json.loads(resp.content)
    
    if data_school['found'] != 0:
        school_lat.append(str(data_school["results"][0]["LATITUDE"]))
        school_long.append(str(data_school["results"][0]["LONGITUDE"]))

        print (str(query_address)+",Lat: "+data_school['results'][0]['LATITUDE'] +" Long: "+data_school['results'][0]['LONGITUDE'])

    else:
        school_long.append('NotFound')
        school_lat.append('NotFound')
        print (str(query_address) + "No Results")

738907,Lat: 1.44298589789841 Long: 103.800304550731
737916,Lat: 1.44589068910993 Long: 103.802398196596
768643,Lat: 1.43368120674913 Long: 103.832924289687
768928,Lat: 1.43623514612253 Long: 103.82998645047
579646,Lat: 1.36073636488519 Long: 103.833075654586
159016,Lat: 1.29141244826495 Long: 103.824359324379
544969,Lat: 1.39099731986818 Long: 103.886918030392
569785,Lat: 1.38419941907925 Long: 103.841411716006
569206,Lat: 1.37424218886593 Long: 103.851429953072
569843,Lat: 1.37839670972543 Long: 103.845885968342
569920,Lat: 1.36909206497026 Long: 103.839041355537
569362,Lat: 1.36735900385008 Long: 103.842600374475
487012,Lat: 1.33062454661529 Long: 103.941062122327
139745,Lat: 1.30328795437358 Long: 103.785262975405
309919,Lat: 1.31945060832386 Long: 103.835698465252
139650,Lat: 1.30244478759311 Long: 103.780249819952
227988,Lat: 1.30937872352186 Long: 103.840789144711
309918,Lat: 1.31871859882709 Long: 103.835345168025
529366,Lat: 1.34857143331006 Long: 103.951673027853
678117,Lat: 1

408940,Lat: 1.32752022209274 Long: 103.901811007466
368051,Lat: 1.34139822375509 Long: 103.87776938227
738927,Lat: 1.4344231166861 Long: 103.773697721298
739110,Lat: 1.43664424769955 Long: 103.776008090454
297754,Lat: 1.34043840661368 Long: 103.839811736775
569948,Lat: 1.37606815582917 Long: 103.835746732105
569384,Lat: 1.36833724573433 Long: 103.835438722324
828867,Lat: 1.39485356081865 Long: 103.90857257736
518798,Lat: 1.37550677054835 Long: 103.934953276155
518901,Lat: 1.3671073515704 Long: 103.949273556351
599986,Lat: 1.33266213599636 Long: 103.783382114018
599986,Lat: 1.33266213599636 Long: 103.783382114018
658965,Lat: 1.35031929565511 Long: 103.741593553753
538786,Lat: 1.37362445234082 Long: 103.889741457901
538785,Lat: 1.37395067790905 Long: 103.888859849008
545079,Lat: 1.38868744166162 Long: 103.890286215943
545080,Lat: 1.39206468503473 Long: 103.890924976943
129956,Lat: 1.30814121110616 Long: 103.769015435345
128806,Lat: 1.31983663784769 Long: 103.761403957527
288683,Lat: 1.33

649036,Lat: 1.34270639669225 Long: 103.687588478681
538784,Lat: 1.37176798317468 Long: 103.88280991785
538789,Lat: 1.3726429004882 Long: 103.883050531748
768611,Lat: 1.43350145509511 Long: 103.838289310553
556108,Lat: 1.3489768083841 Long: 103.868467129954
689100,Lat: 1.39672603497195 Long: 103.751817992962
538720,Lat: 1.37782270940138 Long: 103.885568714764
569868,Lat: 1.37826335261873 Long: 103.841809445345
768675,Lat: 1.42461884491728 Long: 103.829634359313
768679,Lat: 1.43397617334964 Long: 103.834050796658
768516,Lat: 1.42813201188536 Long: 103.828814809222
768610,Lat: 1.43260423932066 Long: 103.838011671738
469623,Lat: 1.33398037894072 Long: 103.932015036296
618654,Lat: 1.34189187145685 Long: 103.720630790899
609558,Lat: 1.34291981622016 Long: 103.740861148879
649406,Lat: 1.34721844938907 Long: 103.722581522719
529393,Lat: 1.35131569890862 Long: 103.950551091373
658712,Lat: 1.34233921440623 Long: 103.760032628302
538884,Lat: 1.35693360328577 Long: 103.890457624477
169485,Lat: 1.2

In [82]:
# Find the 'NotFound' and input manually

In [83]:
school_info['lat']=school_lat
school_info['long']=school_long

In [84]:
school_info[(school_info['lat']=='NotFound')]

,school_name,address,postal_code,mrt_desc,bus_desc,dgp_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,...,nature_code_CO-ED SCHOOL,nature_code_GIRLS' SCHOOL,session_code_FULL DAY,session_code_SINGLE SESSION,mainlevel_code_JUNIOR COLLEGE,mainlevel_code_MIXED LEVELS,mainlevel_code_PRIMARY,mainlevel_code_SECONDARY,lat,long


In [85]:
school_info[(school_info['long']=='NotFound')]

,school_name,address,postal_code,mrt_desc,bus_desc,dgp_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,...,nature_code_CO-ED SCHOOL,nature_code_GIRLS' SCHOOL,session_code_FULL DAY,session_code_SINGLE SESSION,mainlevel_code_JUNIOR COLLEGE,mainlevel_code_MIXED LEVELS,mainlevel_code_PRIMARY,mainlevel_code_SECONDARY,lat,long


## Distance to CBD

In [86]:
df_coordinates = pd.DataFrame({
    'latitude': latitude, #latitude of HDB address
    'longitude': longitude, #longitude of HDB address
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

9327

In [87]:
list_of_lat = df_coordinates['latitude'] # HDB latitude
list_of_long = df_coordinates['longitude'] # HDB longitude
mrt_lat = mrt_location['latitude'] # mrt latitude
mrt_long = mrt_location['longitude'] # mrt longitude

In [88]:
list_of_coordinates = [] # HDB
list_of_mrt_coordinates = [] # MRT
list_of_preschool_coordinates = [] # Pre-school
list_of_school_coordinates = [] # School

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long)) # HDB
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long)) #MRT
for lat, long in zip(preschool_lat, preschool_long):
    list_of_preschool_coordinates.append((lat, long)) # Pre-school
for lat, long in zip(school_lat, school_long):
    list_of_school_coordinates.append((lat, long)) # School

In [89]:
from geopy.distance import geodesic

# Distance from CBD
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

## Distance to nearest MRT

In [90]:
# Distance to nearest MRT

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

In [91]:
list_of_mrt_coordinates[1][1].format

<function str.format>

## Distance to nearest Pre-School

In [92]:
# Distance to nearest Pre-School

list_of_dist_preschool = []
min_dist_preschool = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_preschool_coordinates)):
        list_of_dist_preschool.append(geodesic(origin,list_of_preschool_coordinates[destination]).meters)
    shortest = (min(list_of_dist_preschool))
    min_dist_preschool.append(shortest)
    list_of_dist_preschool.clear()

## Distance to nearest Secondary/Primary School

In [93]:
# Distance to nearest Secondary/Primary School

list_of_dist_school = []
min_dist_school = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_school_coordinates)):
        list_of_dist_school.append(geodesic(origin,list_of_school_coordinates[destination]).meters)
    shortest = (min(list_of_dist_school))
    min_dist_school.append(shortest)
    list_of_dist_school.clear()

## Put all distances into the `df_coordinates`

In [94]:
len(min_dist_mrt), len(min_dist_preschool), len(min_dist_school), len(cbd_dist)

(9327, 9327, 9327, 9327)

In [95]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [96]:
# Put Pre-school and School distance together
df_coordinates['min_dist_preschool'] = min_dist_preschool
df_coordinates['min_dist_school'] = min_dist_school

## Put all distances into `df_dum`

In [97]:
distances = pd.DataFrame(df_coordinates['cbd_dist'])

In [98]:
distances['address'] = df_coordinates['address']
distances['min_dist_preschool'] = df_coordinates['min_dist_preschool']
distances['min_dist_school'] = df_coordinates['min_dist_school']
distances['min_dist_mrt'] = df_coordinates['min_dist_mrt']

In [99]:
distances = distances[['address', 'cbd_dist', 'min_dist_preschool', 'min_dist_school', 'min_dist_mrt']]

In [100]:
df_dum.columns

Index(['date', 'floor_area_sqm', 'lease_commence_date', 'resale_price',
       'month', 'year', 'address', 'remaining_lease_years', 'max_floor_lvl',
       'commercial', 'market_hawker', 'multistorey_carpark',
       'precinct_pavilion', 'own_units', 'flat_type_2 room',
       'flat_type_3 room', 'flat_type_4 room', 'flat_type_5 room',
       'flat_type_executive', 'flat_type_multi-generation',
       'flat_model_adjoined flat', 'flat_model_apartment', 'flat_model_dbss',
       'flat_model_improved', 'flat_model_improved-maisonette',
       'flat_model_maisonette', 'flat_model_model a',
       'flat_model_model a-maisonette', 'flat_model_model a2',
       'flat_model_multi generation', 'flat_model_new generation',
       'flat_model_premium apartment', 'flat_model_premium apartment loft',
       'flat_model_premium maisonette', 'flat_model_simplified',
       'flat_model_standard', 'flat_model_terrace', 'flat_model_type s1',
       'flat_model_type s2', 'storey_range_01 TO 05', 'storey

In [101]:
df_dum = pd.merge(df_dum, distances, on='address')

In [102]:
eda_clean(df_dum)

Dataset Statistics:

Shape of dataframe: (850060, 92)
--------------------------------------
Null values in dataframe: 0
--------------------------------------
% of Null values in dataframe: 0.0%
--------------------------------------
Total duplicate rows: 0
--------------------------------------
% duplicate rows: 0.0%

Variable Types
Columns Count: 
int64      78
float64    12
object      2
dtype: int64


## A little Bit More Feature Engineering

For future use, we will use df_dum and df_coordinates.

In [103]:
df_coordinates.shape, df_dum.shape

((9327, 10), (850060, 92))

In [104]:
#df_coordinates

# 5km distance
df_coordinates['school_5km'] = df_coordinates['min_dist_school'].apply(lambda num: 1 if num<=5000 else 0)
df_coordinates['preschool_5km'] = df_coordinates['min_dist_preschool'].apply(lambda num: 1 if num<=5000 else 0)
df_coordinates['mrt_5km'] = df_coordinates['min_dist_mrt'].apply(lambda num: 1 if num<=5000 else 0)

# 1km distance
df_coordinates['school_1km'] = df_coordinates['min_dist_school'].apply(lambda num: 1 if num<=1000 else 0)
df_coordinates['preschool_1km'] = df_coordinates['min_dist_preschool'].apply(lambda num: 1 if num<=1000 else 0)
df_coordinates['mrt_1km'] = df_coordinates['min_dist_mrt'].apply(lambda num: 1 if num<=1000 else 0)

#df_dum

# 5km distance
df_dum['school_5km'] = df_dum['min_dist_school'].apply(lambda num: 1 if num<=5000 else 0)
df_dum['preschool_5km'] = df_dum['min_dist_preschool'].apply(lambda num: 1 if num<=5000 else 0)
df_dum['mrt_5km'] = df_dum['min_dist_mrt'].apply(lambda num: 1 if num<=5000 else 0)

# 1km distance
df_dum['school_1km'] = df_dum['min_dist_school'].apply(lambda num: 1 if num<=1000 else 0)
df_dum['preschool_1km'] = df_dum['min_dist_preschool'].apply(lambda num: 1 if num<=1000 else 0)
df_dum['mrt_1km'] = df_dum['min_dist_mrt'].apply(lambda num: 1 if num<=1000 else 0)

## Pickle

In [105]:
#Pickle This
df_dum.to_pickle('df_dum.pkl')
df_coordinates.to_pickle('df_coordinates.pkl')